In [2]:
import requests
import time
from datetime import timedelta
from datetime import datetime
import pandas as pd
import urllib.request
from tqdm import tqdm
import json
import asyncio
import numpy as np
import os
from nltk import sent_tokenize
from nltk import word_tokenize

In [2]:
def scrape_and_save_data(pageno):
    city = []
    state = []
    date = []
    lccn = []
    text = []
    title = []

    try:
        # https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext=white+man+ticket&format=json&page=1
        r = s.get(f'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext=white+man+ticket&format=json&page={pageno}', timeout=5)
        r.raise_for_status()  # Check if the request was successful
        data = r.json()

        for item in data.get('items', []):
            city.append(item.get('city', np.nan))
            date.append(item.get('date', np.nan))
            state.append(item.get('state', np.nan))
            lccn.append(item.get('lccn', np.nan))
            text.append(item.get('ocr_eng', np.nan))
            title.append(item.get('title', np.nan))

        df = pd.DataFrame({
            'city': city,
            'state': state,
            'date': date,
            'lCCN': lccn,
            'text': text,
            'title': title
        })

        output_dir =  '/Users/mstudio/Library/CloudStorage/Box-Box/ticket/raw-data/'
        os.makedirs(output_dir, exist_ok=True)
        df.to_csv(f'{output_dir}ticket_{pageno}.csv', index=False)
        time.sleep(30)
    except requests.exceptions.RequestException as e:
        # print(f"Error: {e}")
        failed_page.append(pageno)

failed_page = []
s = requests.Session()

for pageno in tqdm(range(int(np.round(1418 / 20)) + 1)): #6226
    no = pageno+1
    if str(no) in [i.strip('ticket_').strip('.csv') for i in os.listdir('/Users/mstudio/Library/CloudStorage/Box-Box/ticket/raw-data/')]:
        continue
    else:
        scrape_and_save_data(no)


100%|██████████| 72/72 [00:00<00:00, 1120.32it/s]


In [3]:
ticket_path='/Users/mstudio/Library/CloudStorage/Box-Box/ticket/raw-data/'
concat_list=[]
for i in os.listdir(ticket_path):
    if i.startswith('.'):
        continue
    else:
        df = pd.read_csv(ticket_path+i)
        df['filename'] = i.replace('.csv', '')  # Add new column with filename
        concat_list.append(df)

In [9]:
ticket_df=pd.concat(concat_list)
ticket_df.to_csv('/Users/mstudio/Library/CloudStorage/Box-Box/ticket/ticket.csv', index=False)